# Day 5: Inconsistent Data Entry
<br>
Welcome to the final day of the 5-Day Data Challenge. Today, we're are topic is to learn how to clean up inconsistent text entries.
Here's what we're going to do today:

   - Do some preliminary text pre-processing
   - Use fuzzy matching to correct inconsistent data entry

Let's get started!

In [44]:
import pandas as pd
import numpy as np

#helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# set seed for reproducibility
np.random.seed(0)

When I tried to read in the PakistanSuicideAttacks Ver 11 (30-November-2017).csvfile the first time, I got a character encoding error, so I'm going to quickly check out what the encoding should be...

In [45]:
# look at the first hundred thousand bytes to guess the character encoding
with open("PakistanSuicideAttacks Ver 11 (30-November-2017).csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))

# check what the character encoding might be
print(result)

{'encoding': 'ISO-8859-2', 'confidence': 0.7479402284082854}


In [46]:
suicide_attacks = pd.read_csv("PakistanSuicideAttacks Ver 11 (30-November-2017).csv", encoding = "ISO-8859-2")

Now we got the data read in the correct encoding. We should start by first looking at the data and get familiar with it.

## Preliminary text pre-processing
<br>
For this exercise, I'm interested in cleaning up the "City" column to make sure there's no data entry inconsistencies in it. We could go through and check each row by hand, of course, and hand-correct inconsistencies when we find them. There's a more efficient way to do this though!

In [47]:
# get all unique cities in 'City' column
cities = suicide_attacks['City'].unique()

# sort them alphabetically 
cities.sort()
cities

array(['ATTOCK', 'Attock ', 'Bajaur Agency', 'Bannu', 'Bhakkar ', 'Buner',
       'Chakwal ', 'Chaman', 'Charsadda', 'Charsadda ', 'D. I Khan',
       'D.G Khan', 'D.G Khan ', 'D.I Khan', 'D.I Khan ', 'Dara Adam Khel',
       'Dara Adam khel', 'Fateh Jang', 'Ghallanai, Mohmand Agency ',
       'Gujrat', 'Hangu', 'Haripur', 'Hayatabad', 'Islamabad',
       'Islamabad ', 'Jacobabad', 'KURRAM AGENCY', 'Karachi', 'Karachi ',
       'Karak', 'Khanewal', 'Khuzdar', 'Khyber Agency', 'Khyber Agency ',
       'Kohat', 'Kohat ', 'Kuram Agency ', 'Lahore', 'Lahore ',
       'Lakki Marwat', 'Lakki marwat', 'Lasbela', 'Lower Dir', 'MULTAN',
       'Malakand ', 'Mansehra', 'Mardan', 'Mohmand Agency',
       'Mohmand Agency ', 'Mohmand agency', 'Mosal Kor, Mohmand Agency',
       'Multan', 'Muzaffarabad', 'North Waziristan', 'North waziristan',
       'Nowshehra', 'Orakzai Agency', 'Peshawar', 'Peshawar ', 'Pishin',
       'Poonch', 'Quetta', 'Quetta ', 'Rawalpindi', 'Sargodha',
       'Sehwan town',

Just from this single column we can spot several data entry inconsistencies. For instance, we have 'ATTOCK' and 'Attock ', and 'Lahore' and 'Lahore '.
<br>
The first thing we should do is make everything lower case to fix some of these inconsistencies and remove any white spaces in the beginning and end of cells. These are probably the most common errors you encounter when dealing with text data.

In [48]:
# convert to lower
suicide_attacks['City'] = suicide_attacks['City'].str.lower()

#remove trailing white spaces
suicide_attacks['City'] = suicide_attacks['City'].str.strip()

Next, we're going to tackle more difficult inconsistencies.

## Use fuzzy matching to correct inconsistent data entry
<br>
Okay, let's look at the city column again and see if there's more cleaning we need to do.


In [49]:
# get all unique cities from 'City' column
cities = suicide_attacks['City'].unique()

# sort alphabetically
cities.sort()
cities

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd. i khan', 'd.g khan', 'd.i khan',
       'dara adam khel', 'fateh jang', 'ghallanai, mohmand agency',
       'gujrat', 'hangu', 'haripur', 'hayatabad', 'islamabad',
       'jacobabad', 'karachi', 'karak', 'khanewal', 'khuzdar',
       'khyber agency', 'kohat', 'kuram agency', 'kurram agency',
       'lahore', 'lakki marwat', 'lasbela', 'lower dir', 'malakand',
       'mansehra', 'mardan', 'mohmand agency',
       'mosal kor, mohmand agency', 'multan', 'muzaffarabad',
       'north waziristan', 'nowshehra', 'orakzai agency', 'peshawar',
       'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'],
      dtype=

We can see that inconsistencies still do exist: 'd. i khan', 'd.i khan' should probably be the same('d.g khan' is actually a seperate city, so I shouldn't combine those).
<br>
<br>

We are going to use the fuzzywuzzy package to help identify which string are closest to each other. 
<br>
Fuzzywuzzy returns a ratio given two strings. The closer the ratio is to 100, the smaller the edit distance between the two strings. Here, we're going to get the ten strings from our list of cities that have the closest distance to "d.i khan".

In [50]:
# get top 10 closest matches to 'd.i khan'
matches = fuzzywuzzy.process.extract('d.i khan', cities, limit = 10, scorer = fuzzywuzzy.fuzz.token_sort_ratio)

# take a look
matches

[('d. i khan', 100),
 ('d.i khan', 100),
 ('d.g khan', 88),
 ('khanewal', 50),
 ('sudhanoti', 47),
 ('hangu', 46),
 ('kohat', 46),
 ('dara adam khel', 45),
 ('chaman', 43),
 ('mardan', 43)]

Since we don't want to replace "d.g khan" with "d.i khan", let's replace all rows in our City column that have a ratio of > 90 with "d. i khan". 
To do this, let's write a function:


In [51]:
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get list of unique strings
    strings = df[column].unique()
    
    # get top 10 closest matches to 'd.i khan'
    matches = fuzzywuzzy.process.extract(string_to_match, strings, limit = 10, scorer = fuzzywuzzy.fuzz.token_sort_ratio)
    
    # only get matches with ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    
    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)
    
    # replace all rows with close matches with input matches
    df.loc[rows_with_matches,column] = string_to_match
    
    # let us know when function is done
    print("All done.")
    

Now that we have a function, we can put it to the test!

In [52]:
# Use function to replace 'd. i khan' with 'd.i khan'
replace_matches_in_column(df =suicide_attacks, column = 'City',string_to_match= 'd.i khan')

All done.


Let's check if our function actually worked. 

In [53]:
# get unique cities from 'City' column
cities = suicide_attacks['City'].unique()

# sort
cities.sort()
cities

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd.g khan', 'd.i khan', 'dara adam khel',
       'fateh jang', 'ghallanai, mohmand agency', 'gujrat', 'hangu',
       'haripur', 'hayatabad', 'islamabad', 'jacobabad', 'karachi',
       'karak', 'khanewal', 'khuzdar', 'khyber agency', 'kohat',
       'kuram agency', 'kurram agency', 'lahore', 'lakki marwat',
       'lasbela', 'lower dir', 'malakand', 'mansehra', 'mardan',
       'mohmand agency', 'mosal kor, mohmand agency', 'multan',
       'muzaffarabad', 'north waziristan', 'nowshehra', 'orakzai agency',
       'peshawar', 'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'],
      dtype=object)

Excellent! Now we only have "d.i khan" in our dataframe and we didn't have to change anything by hand. We can use the same function to fix the same inconsistency we see with 'kuram agency' and 'kurram agency' as well. 
<br>
And that's it for today. Thanks for following along with the 5-Day Data challenge.